SCARF embeding example

In [25]:
pip install git+https://github.com/wuphwu/pytorch-scarf.git

  Cloning https://github.com/wuphwu/pytorch-scarf.git to /tmp/pip-req-build-nr3_41cw
  Running command git clone --filter=blob:none --quiet https://github.com/wuphwu/pytorch-scarf.git /tmp/pip-req-build-nr3_41cw
  Resolved https://github.com/wuphwu/pytorch-scarf.git to commit e7f4925320a7650a6c3d0596fe0c5817f0c1e0cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# 載入資料

In [26]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
hassan06_nslkdd_path = kagglehub.dataset_download('hassan06/nslkdd')

print('Data source import complete.')
print(f"資料集下載路徑：{hassan06_nslkdd_path}")


Data source import complete.
資料集下載路徑：/kaggle/input/nslkdd


In [27]:
import os
import pandas as pd
# 使用下載的路徑來建構檔案的完整路徑
file_path = os.path.join(hassan06_nslkdd_path, 'KDDTrain+.txt')

df_0 = df_0 = pd.read_csv(file_path, sep=',', header=None, encoding='latin-1')
df= df_0.copy()
df.head() # 顯示資料集前五行

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [28]:
# 定義資料集的欄位名稱
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'level'])

df.columns = columns

In [29]:
# 區分類別型、連續型欄位
category_cols = df.select_dtypes(include='object').drop(["attack"], axis=1).columns
print(category_cols)
continuous_cols = df.select_dtypes(exclude='object').drop(["level"], axis=1).columns
print(continuous_cols)

Index(['protocol_type', 'service', 'flag'], dtype='object')
Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate'],
      dtype='object')


# y lable one-hot encoding
因應SCARF執行上的需要，進行attack欄位編碼,避免字串

In [30]:
# 用LabelEncoder()把y的資料轉換成整數型態
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
clm=['attack']
for x in clm:
    df[x]=le.fit_transform(df[x])

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scarf.dataset import SCARFDataset

## 2.3.2 Frequency Encoding(要分割完測試資料再做)

就是我們把Categorical Feature裡面每個類別出現的數量，當成他的數值。

In [33]:
!pip install category_encoders
from category_encoders import CountEncoder

In [34]:
# use target encoding to encode categorical features
categorical_features_to_encode = ['protocol_type', 'service', 'flag']
enc = CountEncoder(cols=categorical_features_to_encode)

# transform the datasets
X_train = enc.fit_transform(train_data)
X_test = enc.transform(test_data)

In [32]:
seed = 1234
data = df.copy()
data, target = data.drop(['level','attack'], axis=1), data["attack"]
train_data, test_data, train_target, test_target = train_test_split(
    data, target, test_size=0.2, stratify=target, random_state=seed
)


In [36]:
pip install torch

In [44]:
from scarf.loss import NTXent
from scarf.model import SCARF
from torch import torch
from torch.optim import Adam
from torch.utils.data import DataLoader


# preprocess your data and create your pytorch dataset


# train the model
batch_size = 128
epochs = 5000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

import numpy as np # Import the numpy library
# Check for features where features_low >= features_high
problematic_features_indices = np.where(train_ds.features_low >= train_ds.features_high)[0]

if len(problematic_features_indices) > 0:
    print("Found problematic features where features_low >= features_high:")
    for idx in problematic_features_indices:
        print(f"Feature: {train_ds.columns[idx]}, Low: {train_ds.features_low[idx]}, High: {train_ds.features_high[idx]}")
    # 若要繼續執行，您可能需要從您的資料集中移除這些欄位，
    # 在建立 SCARFDataset 之前。
    # 例如：
    problematic_columns = [train_ds.columns[i] for i in problematic_features_indices]
    print(f"Dropping problematic columns: {problematic_columns}")
    X_train = X_train.drop(columns=problematic_columns)
    X_test = X_test.drop(columns=problematic_columns)
    # 重新建立 SCARFDataset 和 DataLoader
    train_ds = SCARFDataset(X_train.to_numpy(), train_target.to_numpy(), columns=X_train.columns)
    test_ds = SCARFDataset(X_test.to_numpy(), test_target.to_numpy(), columns=X_test.columns)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    print("重新建立 SCARFDataset 和 DataLoader，已移除有問題的欄位。")
else:
    print("所有特徵的 features_low 皆小於 features_high。繼續進行模型初始化。")


model = SCARF(
    input_dim=train_ds.shape[1],
    dim_hidden_encoder=16,
    num_hidden_encoder=6,
    dim_hidden_head=6,
    num_hidden_head=6,
    features_low=train_ds.features_low,
    features_high=train_ds.features_high,
    corruption_rate=0.6,
    dropout=0.1
).to(device)

optimizer = Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
ntxent_loss = NTXent()

loss_history = []

for epoch in range(1, epochs + 1):
    epoch_loss = 0.0
    for x in train_loader:
        x = x.to(device)
        emb_anchor, emb_positive = model(x)

        loss = ntxent_loss(emb_anchor, emb_positive)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        epoch_loss += loss.item()

    loss_history.append(epoch_loss)

    if epoch % 100 == 0:
       print(f"epoch {epoch}/{epochs} - loss: {loss_history[-1]:.4f}")

所有特徵的 features_low 皆小於 features_high。繼續進行模型初始化。
epoch 100/5000 - loss: 4365.3933
epoch 200/5000 - loss: 4365.3933
epoch 300/5000 - loss: 4365.3933
epoch 400/5000 - loss: 4365.3933


KeyboardInterrupt: 